# Проектная работа: SQL 

### Инструкция:

- Описание цели исследования;
- Загрузка и изучение данных;
- Исследовательский анализ;
- Вывод.

**Описание:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Задача** — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Задания

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Загрузка и изучение данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# выведем первые строки таблицы books
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# выведем первые строки таблицы authors
query = '''SELECT * 
           FROM authors 
           LIMIT 5
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# выведем первые строки таблицы publishers
query = '''SELECT * 
           FROM publishers 
           LIMIT 5
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# выведем первые строки таблицы ratings
query = '''SELECT * 
           FROM ratings 
           LIMIT 5
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# выведем первые строки таблицы reviews
query = '''SELECT * 
           FROM reviews 
           LIMIT 5
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Исследовательский анализ данных

In [8]:
# напишем функцию для выполнения запросов
def conclusion(sql):
    return pd.io.sql.read_sql(sql, con = engine) 
query_books = '''
        SELECT * FROM books
        '''
books = conclusion(query_books)

query_authors = '''
                SELECT * FROM authors
                '''
authors = conclusion(query_authors)

query_publishers = '''
                   SELECT * FROM publishers
                   '''
publishers = conclusion(query_publishers)

query_ratings = '''
                SELECT * FROM ratings
                '''
ratings = conclusion(query_ratings)

query_reviews = '''
                SELECT * FROM reviews
                '''
reviews = conclusion(query_reviews)

### 1. Необходимо посчитать, сколько книг вышло после 1 января 2000 года:

In [9]:
books_1 = '''
            SELECT 
                COUNT(book_id) AS books_cnt 
            FROM 
                books
            WHERE 
                publication_date > '2000-01-01'
            '''
conclusion(books_1)

,books_cnt
0,819


819 книг было выпущено после с 1 января 2000 года

### 2. Для каждой книги необходимо посчитать количество обзоров и среднюю оценку:

In [10]:
review_evaluation = '''                              
                      SELECT
                            books.title AS book_title,
                            books.book_id AS book_id,
                            AVG(ratings.rating) AS avg_rating,
                            COUNT(DISTINCT reviews.review_id) AS count_reviews -- DISTINCT только уникальные по столбцу

                      FROM
                          books

                      INNER JOIN ratings ON ratings.book_id = books.book_id
                      LEFT JOIN reviews ON reviews.book_id = books.book_id
                    

                      GROUP BY
                              books.title,
                              books.book_id

                      ORDER BY
                              count_reviews DESC;
                   ''' 
conclusion(review_evaluation).round(2)

,book_title,book_id,avg_rating,count_reviews
0,Twilight (Twilight #1),948,3.66,7
1,Water for Elephants,963,3.98,6
2,The Glass Castle,734,4.21,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.41,6
4,The Curious Incident of the Dog in the Night-Time,695,4.08,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.67,0
996,The Natural Way to Draw,808,3.00,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.00,0
998,Essential Tales and Poems,221,4.00,0


По количеству обзоров лидирует «Сумерки» книга 1 с общим числом обзоров 7.

### 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры:

In [11]:
publishing_house = '''
                    SELECT  
                        publishers.publisher AS pbl_house, 
                        COUNT(books.book_id) AS number_books
                    FROM
                        publishers
                        LEFT JOIN books ON books.publisher_id = publishers.publisher_id
                    WHERE
                        books.num_pages > 50  
                    GROUP BY
                        pbl_house
                    ORDER BY
                        number_books DESC
                    LIMIT 1
                    '''
conclusion(publishing_house)   

,pbl_house,number_books
0,Penguin Books,42


**Penguin Books** это британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном., оно выпустило 42 книги в базе.

### 4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками:

In [12]:
high_author = '''
                SELECT 
                    SUBQ.author AS author,  
                    AVG(SUBQ.rating_avg) AS rating 
                FROM (
                    SELECT
                        authors.author AS author,
                        books.book_id AS id_book, 
                        AVG(ratings.rating) AS rating_avg,
                        COUNT(ratings.rating) AS rating_cnt
                    FROM 
                        authors
                        INNER JOIN books ON books.author_id = authors.author_id
                        INNER JOIN ratings ON ratings.book_id = books.book_id
                    GROUP BY 
                        author,
                        id_book
                        HAVING
                        COUNT(ratings.rating) >= 50) AS SUBQ
                GROUP BY
                    author
                ORDER BY 
                    rating DESC
                LIMIT 10
                '''
conclusion(high_author).round(2)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
9,Dan Brown,3.75


Это Мэри Грандпрег.

### 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [13]:
review = '''
            SELECT 
                  ROUND(AVG(SUBQ.reviews_cnt), 2)
            FROM(
                SELECT
                      username,
                      COUNT(review_id) AS reviews_cnt
                FROM
                    reviews
                WHERE 
                    username IN (
                                 SELECT
                                       username
                                 FROM
                                     ratings
                                 GROUP BY
                                         username 
                                 HAVING 
                                       COUNT(ratings.rating_id) > 48)
                                 GROUP BY
                                         username) AS SUBQ
            '''
conclusion(review)

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - их 24.

# Вывод:

1. Посчитали и узнали, что 819 книг было выпущено после с 1 января 2000 года.

2. Посчитали количество обзоров и среднюю оценку для каждой книги. По количеству обзоров лидирует «Сумерки» книга 1 с общим числом обзоров 7.

3. Определили издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books это британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном., оно выпустило 42 книги в базе.

4. Определили автора с самой высокой средней оценкой книг это Мэри Грандпрег.

5. Посчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок - их 24.